In [2]:
%load_ext autoreload
%autoreload 2

# Get parent root directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# from rocket_euler import Rocket
from src.pos_rocket_vis import *
from NonLinearMPC_7.nmpc_land import NmpcCtrl
import numpy as np

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir,"rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7 # Do not change!!!

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [7]:
sim_time = 4 # simulation length in seconds
x0 = np.array([0, 0, 0, np.deg2rad(30), 0, 0, 0, 0, 0, 3, 2, 10]) # w, phi, v, p
x_ref = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3])
xs, us = rocket.trim(x_ref)
print("Linearization around a steady state:")
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)

H = 0.5 # horizon time 
nmpc = NmpcCtrl(rocket, H, xs, us)

Linearization around a steady state:
x_ref =  [0 0 0 0 0 0 0 0 0 1 0 3]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
us =  [ 0.          0.         56.66666667  0.        ]


In [8]:
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(nmpc, sim_time, H, x0)

Simulating time 0.00
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 141.00us (  5.88us) 140.57us (  5.86us)        24
       nlp_g  | 578.00us ( 24.08us) 556.48us ( 23.19us)        24
  nlp_grad_f  | 235.00us (  9.40us) 232.86us (  9.31us)        25
  nlp_hess_l  |   1.80ms ( 78.22us)   1.96ms ( 85.30us)        23
   nlp_jac_g  |   1.29ms ( 51.76us)   1.29ms ( 51.55us)        25
       total  |  80.35ms ( 80.35ms)  95.55ms ( 95.55ms)         1


ValueError: could not broadcast input array from shape (12,41) into shape (12,11)

In [ ]:
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol)
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs)